In [1]:
from rl4co.envs.routing import TSPEnv, TSPGenerator
from rl4co.models import DeepACOPolicy, DeepACO
from rl4co.utils import RL4COTrainer


from lightning.pytorch.callbacks import ModelCheckpoint, RichModelSummary
from lightning.pytorch.loggers import WandbLogger

from rl4co.models.zoo.deepaco.antsystem import AntSystem
from rl4co.models.zoo.hdgaco.FocusedACO import FocusedACO

/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Instantiate generator and environment
num_loc = 50
generator = TSPGenerator(num_loc=num_loc, loc_distribution="uniform")
env = TSPEnv(generator)

In [ ]:

policy = DeepACOPolicy(env_name=env.name, 
                     aco_class=FocusedACO,
                     k_sparse=num_loc, 
                     train_with_local_search=True,
                     aco_kwargs={"use_local_search": True})
model = DeepACO(env, 
              policy, 
              batch_size=512, 
              train_data_size=10_000,
              val_data_size=1_000,
              val_batch_size=512,
              test_data_size=1_000, 
              optimizer_kwargs={"lr": 1e-4}
              )

logger = WandbLogger(project="hdgaco", name="tsp_50_deepaco_faco")

/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.


In [4]:
checkpoint_callback = ModelCheckpoint(  dirpath="checkpoints/gfacs_mmas_{num_loc}", # save to checkpoints/
                                        filename="epoch_{epoch:03d}",  # save as epoch_XXX.ckpt
                                        save_top_k=1, # save only the best model
                                        save_last=True, # save the last model
                                        monitor="val/reward", # monitor validation reward
                                        mode="max") # maximize validation reward


rich_model_summary = RichModelSummary(max_depth=3)

callbacks = [checkpoint_callback, rich_model_summary]


In [5]:
trainer = RL4COTrainer(
    max_epochs=20,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=callbacks,
)

Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model)


/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:45: Attribute 'policy' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['policy'])`.
wandb: Currently logged in as: shoraaa (shoraaa-vnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /home/shora/Research/rl4co/checkpoints/gfacs_mmas_{num_loc} exists and is not empty.
val_file not set. Generating dataset instead
test_file not set. Generating dataset instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                             ┃ Type                     ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ env                              │ TSPEnv                   │      0 │ train │
│ 1 │ policy                           │ DeepACOPolicy            │  332 K │ train │
│ 2 │ policy.encoder                   │ NARGNNEncoder            │  332 K │ train │
│ 3 │ policy.encoder.init_embedding    │ TSPInitEmbedding         │    192 │ train │
│ 4 │ policy.encoder.edge_embedding    │ TSPEdgeEmbedding         │    128 │ train │
│ 5 │ policy.encoder.graph_network     │ GNNEncoder               │  315 K │ train │
│ 6 │ policy.encoder.heatmap_generator │ EdgeHeatmapGenerator     │ 16.7 K │ train │
│ 7 │ policy.decoder                   │ NonAutoregressiveDecoder │      0 │ train │
│ 8 │ baseline                         │ NoBaseline               │      0 │ train │
└───┴──────────────────────────────────┴──────────────────────────┴────────┴───────┘

Trainable params: 332 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 332 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 168                                                                                         
Modules in eval mode: 0

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 19: 100%|██████████| 20/20 [00:31<00:00,  0.64it/s, v_num=37pl, train/reward=-7.22, train/loss=-0.19, val/reward=-5.67] 

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 20/20 [00:31<00:00,  0.64it/s, v_num=37pl, train/reward=-7.22, train/loss=-0.19, val/reward=-5.67]


In [7]:
trainer.test(model)

val_file not set. Generating dataset instead
test_file not set. Generating dataset instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/shora/Research/rl4co/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 2/2 [00:10<00:00,  0.19it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/reward        │     -5.68765115737915     │
└───────────────────────────┴───────────────────────────┘

[{'test/reward': -5.68765115737915}]